# Clean sleep data

In [1]:
import pandas as pd
from zipfile import ZipFile

save_to_db = True

In [2]:
with ZipFile('../data/sleep-export.zip') as my_zip:
    with my_zip.open('sleep-export.csv', 'r') as f:
        lines = list(map(lambda x: x.decode('utf-8'), f.readlines()))

The file format is a modification of a csv file created by [Sleep as Android](https://docs.sleep.urbandroid.org/devs/csv.html). It needs to be parsed. There are three types of data to extract.

- Generic information for each sleep
- All [events](https://docs.sleep.urbandroid.org/devs/sleepcloud_api.html#event-labels) during a sleeping period
- Accelerometer information during a sleeping period

Initially we create two dataframes, one including the generic information and the other one the event and accelerometer information. Then, we split the latter into the event and accelerometer dataframes.

In [3]:
def get_sleeps(data: list[str]) -> list[list[str]]:
    while len(data) > 0:
        i = 2 if data[2].startswith('Id') else 3

        sleep, data = data[:i], data[i:]
        yield [list(map(lambda y: y.replace('"', '')
                        .replace('\n', ''), x.split(','))) for x in sleep]

In [4]:
sleep_data = []
event_data = []
for sleep in get_sleeps(lines):
    sleep_data.append(sleep[1][:15])

    for time, mov_value in zip(sleep[0][15:], sleep[1][15:]):
        event_data.append([sleep[1][0], time, mov_value])


headers = sleep[0][:15]

sleep_df = pd.DataFrame(sleep_data, columns=headers)
event_df = pd.DataFrame(event_data, columns=['Id', 'Time', 'Value'])
movement_df = event_df[event_df['Time'] != 'Event']
event_df = event_df[event_df['Time'] == 'Event'][['Id', 'Value']].rename(
    columns = {'Value': 'Event'}
)

Clean sleep data

In [5]:
for col in ('From', 'To', 'Sched'):
    sleep_df[col] = pd.to_datetime(sleep_df[col], format='%d. %m. %Y %H:%M')

for col in ('Id', 'Hours', 'Rating', 'DeepSleep', 'Framerate', 'Snore', 'Noise',
            'Cycles', 'LenAdjust'):
    sleep_df[col] = pd.to_numeric(sleep_df[col])

sleep_df = sleep_df.rename(
    columns={old: old.lower() for old in sleep_df.columns}
)
sleep_df = sleep_df.rename(columns={'from': 'sleep_from', 'to': 'sleep_to'})

Clean accelerometer data

In [6]:
for col in ('Id', 'Value'):
    movement_df[col] = pd.to_numeric(movement_df[col])

movement_df['Time'] = pd.to_datetime(movement_df['Time'], format='%H:%M').dt.time

movement_df = movement_df.rename(
    columns={old: old.lower() for old in movement_df.columns}
    )

event_df['Id'] = pd.to_numeric(event_df['Id'])

event_df = event_df.rename(
    columns={old: old.lower() for old in event_df.columns}
    )

Save clean data

In [7]:
sleep_df.to_csv('../data/clean/sleeps.csv', index=False)
movement_df.to_csv('../data/clean/movements.csv', index=False)
event_df.to_csv('../data/clean/events.csv', index=False)

There is more cleaning to be done. 

For instance, I only used the comment feature of the app once when I downloaded and it has an irrelevant comment in Catalan, but the app automatically adds different tags that can be extracted into a new table of its own.

The events dataframe includes all events that happen in a night with a timestamp. Again, this can be parsed and utilized.

## Put raw data in SQLite3 database

Since the idea is to use dbt for the data transformation until we get analysis ready tables, the data will also be added to a locally managed SQLite3 database.

In [8]:
import sqlite3

con = sqlite3.connect('../database/raw.db')

In [9]:
con.execute('DROP TABLE IF EXISTS sleeps;')
con.execute('DROP TABLE IF EXISTS movements;')
con.execute('DROP TABLE IF EXISTS events;')

In [10]:
sleep_df.to_sql('sleeps', con, index=False)
movement_df.to_sql('movements', con, index=False)
event_df.to_sql('events', con, index=False)

228669